In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
import collections
import matplotlib.pylab as plt


## Read Data

In [2]:
titles_df = pd.read_csv('../Data/titles_performance.csv')

header_lst = ['title', 'date_range', 'year', 'rank', 'weekend_gross', 'gross_to_date', "week_no_release"]
titles_df.columns = header_lst

titles_df['title'] = np.where(titles_df['title'] == 'Night school', 'Night School', titles_df['title'])
titles_df['title'] = np.where(titles_df['title'] == 'Avengers: Infinty War',\
                              'Avengers Infinity War', titles_df['title'])


In [3]:
titles_df.head()

#date range from Thur - Sun

,title,date_range,year,rank,weekend_gross,gross_to_date,week_no_release
0,Incredibles 2,Jun. 14–17,2018,1,"$7,898,856","$7,898,856",1
1,Incredibles 2,Jun. 21–24,2018,2,"$4,868,158","$14,708,509",2
2,Incredibles 2,Jun. 28–Jul. 1,2018,2,"$3,253,530","$19,313,351",3
3,Incredibles 2,Jul. 5–8,2018,2,"$2,940,132","$24,578,641",4
4,Incredibles 2,Jul. 12–15,2018,2,"$2,586,378","$29,501,312",5


## Clean Date

Create another column in df to store date as 'YYYY/MM/DD'.

Note that date_range covers all dates from Thur-Sun. We will represent the data by its 1st date. 
i.e date_range = 'Jun. 14-17', new column stores '2018/06/14'

In [4]:
def get_month(month):
    try:
        if month == 'Jan': return '01'
        elif month =='Feb': return '02'
        elif month =='Mar': return '03'
        elif month =='Apr': return '04'
        elif month =='May': return '05'
        elif month =='Jun': return '06'
        elif month =='Jul': return '07'
        elif month =='Aug': return '08'
        elif month =='Sep': return '09'
        elif month =='Oct': return '10'
        elif month =='Nov': return '11'
        elif month =='Dec': return '12'
    
    except Exception as e:
        print(e)

In [5]:
#create new column 

titles_df['format_date'] = ''
for index, row in titles_df.iterrows():
    #get exact date
    dash_index = re.search("–", row['date_range']).start() 
    space_index = re.search(" ", row['date_range']).start()
    date_num = row['date_range'][space_index+1: dash_index]
    date_val = date_num if len(date_num)== 2 else "0"+str(date_num)
    
    #get month
    month = get_month(row['date_range'][:3])
    
    new_date = str(row['year']) + '-' + str(month) + '-' + str(date_val)  
    titles_df.loc[index, 'format_date'] = new_date
    
print("number of records we have before preprocessing is : " + str(len(titles_df)))
titles_df.head(5)


number of records we have before preprocessing is : 253


,title,date_range,year,rank,weekend_gross,gross_to_date,week_no_release,format_date
0,Incredibles 2,Jun. 14–17,2018,1,"$7,898,856","$7,898,856",1,2018-06-14
1,Incredibles 2,Jun. 21–24,2018,2,"$4,868,158","$14,708,509",2,2018-06-21
2,Incredibles 2,Jun. 28–Jul. 1,2018,2,"$3,253,530","$19,313,351",3,2018-06-28
3,Incredibles 2,Jul. 5–8,2018,2,"$2,940,132","$24,578,641",4,2018-07-05
4,Incredibles 2,Jul. 12–15,2018,2,"$2,586,378","$29,501,312",5,2018-07-12


In [6]:
all_titles = titles_df.title.unique()
print("total number of unique movies is: " + str(len(all_titles)))
print("List of unique movies")
print(all_titles)

total number of unique movies is: 19
List of unique movies
['Incredibles 2' 'Beauty and the Beast' 'The Greatest Showman'
 'Black Panther' 'Avengers Infinity War' 'Jurassic World' 'Wonder Woman'
 'Star Wars: The Last Jedi' 'Coco' 'Overlord' 'Dunkirk' 'Night School'
 'The Grinch' 'The Predator' 'The Meg' 'Occupation' 'Ladies in Black'
 'Sweet Country' 'Lion']


In [7]:
#Change data type of gross_to_date to integers. 
titles_df['gross_to_date'] = titles_df['gross_to_date'].str.replace('$', '')
titles_df['gross_to_date'] = titles_df['gross_to_date'].str.replace(',', '')
titles_df['gross_to_date'].astype(int)

titles_df['weekend_gross'] = titles_df['weekend_gross'].str.replace('$', '')
titles_df['weekend_gross'] = titles_df['weekend_gross'].str.replace(',', '')
titles_df['weekend_gross'].fillna(0).astype(int)

titles_df.head(5)

,title,date_range,year,rank,weekend_gross,gross_to_date,week_no_release,format_date
0,Incredibles 2,Jun. 14–17,2018,1,7898856,7898856,1,2018-06-14
1,Incredibles 2,Jun. 21–24,2018,2,4868158,14708509,2,2018-06-21
2,Incredibles 2,Jun. 28–Jul. 1,2018,2,3253530,19313351,3,2018-06-28
3,Incredibles 2,Jul. 5–8,2018,2,2940132,24578641,4,2018-07-05
4,Incredibles 2,Jul. 12–15,2018,2,2586378,29501312,5,2018-07-12


## Data Imputation and Smoothing

#### Data Imputation
: Data is not recorded consistently every week. We will replace missing values with averages.

#### Smoothing
: Gross-to-date is cumulative, hence it should be strictly increasing. However, many of the more popular films like 'Avengers', 'Star Wars', we observe that values are not monotonic increasing, and this should not be ignored. I will smooth the curve to ensure that it is strictly increasing. 

We will create a new table, with adjusted gross_to_date

#### Get weekly earnings
: Create new column 'week_gross'

In [8]:
#some functions we will need later
def sort_dictionary(dic):
    """
    PARAM: dictionary
    OUTPUT: sorted dictionary.
    dont use OrderedDict
    """
    sorted_dic = dict()
    for key in sorted(dic.keys()):
        sorted_dic[key] = dic[key]

    return sorted_dic

def adjust_dictionary(dic):
    """
    PARAM: dictionary
    OUTPUT: all gross to date are strictly increasing. 
    Interpolation does not work since we want it to increase only.

    """
    sorted_dic = dict()
    gross_lst = [dic[key][1] for key in list(dic.keys())]
    gross_lst.sort()
    for index, key in enumerate(sorted(dic.keys())):
        val = dic[key]
        sorted_dic[key] = (dic[key][0], gross_lst[index])

    return sorted_dic



In [9]:
def show_plot(new_dict, og_dict):
    """
    PARAM: 
    | new_dict = dictionary containing gross-to-date every week, with adjusted values
    | og_dict = dictionary containing gross-to-date from original dataset, with missing values
    
    OUTPUT: plot both dictionary values for comparison 
    """
    fig, ax = plt.subplots(1,1,figsize=(9,6))

    #Plot original data, with some missing data and not monotonic increasing
    x2 = [i for i in range(1, len(new_dict)+1)]
    y2 = [og_dict[i][1] if i in list(og_dict.keys()) else None for i in range(1, len(new_dict)+1)]
    plt.plot(x2, y2,linestyle='-', marker='o',color='green', label='original')

    #Plot adjusted values
    lists = sorted(new_dict.items()) # sorted by key, return a list of tuples
    x = list(new_dict.keys())
    y = [new_dict[i][1] for i in new_dict.keys()]

    plt.plot(x, y, linestyle='-', marker='o',color='blue', label='adjusted')
    plt.legend(loc='lower right')
    plt.ylabel('Gross-to-date $')
    plt.xlabel('Weeks after release')
    plt.title('Gross-to-date Overtime')
    plt.show()
    
    return
    

In [10]:
def get_adj_val(max_weeks, temp_dict):
    """
    PARAM:
    | max_weeks  - Number of weeks worthg of data
    | temp_dict - dictionary containing gross to date values in original dataset
    
    OUTPUT: modified dictionary, with missing values included. 
    
    """
    #assign values to weeks with missing data
    for num in range(2, max_weeks):
        curr_index = num

        if curr_index not in temp_dict.keys():
            #print("index that was missing " + str(num))

            prev_date = temp_dict[num-1][0] #get date
            prev_date = datetime.strptime(prev_date, "%Y-%m-%d")
            curr_date = prev_date + timedelta(days=7)

            prev_val = temp_dict[num-1][1]
            next_index = curr_index+1

            #keep finding the next week with data avai
            while next_index not in temp_dict.keys():
                next_index +=1

            next_val = int(temp_dict[next_index][1])
            diff_val = next_val - prev_val
            diff_index = next_index - (curr_index -1)
            add_val = diff_val/diff_index
            curr_val = prev_val + add_val
            curr_date = curr_date.strftime('%Y-%m-%d')

            #add new values to the dictionary
            temp_dict[num] = (str(curr_date), int(curr_val))
    
    temp_dict = sort_dictionary(temp_dict) #just in case
    
    return temp_dict

In [11]:
def add_weekly_gross(df):
    """
    PARAM: df, we will add in that weeks gross revenue
    OUTPUT: updated df with 2 new columns : shift, week_gross
    """
    df['shift'] = ''
    df['shift'] = df.adj_gtd.shift(1)
    df['shift'].fillna(0, inplace=True)
    df['week_gross'] = df['adj_gtd'] - df['shift']
    
    return df


In [12]:
def get_movie_df(movie, df):
    max_weeks = df.week_no_release.max() #maximum number of weeks we have data
    
    #create temporary dictionary
    temp_dict = dict((row['week_no_release'], (str(row['format_date']), int(row['gross_to_date']))) for index, row in df.iterrows())
    og_dict = temp_dict.copy() #original copy
    
    temp_dict = sort_dictionary(temp_dict)
    temp_dict = adjust_dictionary(temp_dict)
    temp_dict = get_adj_val(max_weeks,temp_dict)
    
    
    ########
    # Plot #
    ########
    #show_plot(temp_dict, og_dict) #remove comment to see graphs before and after smoothing
    
    #Add data to df
    new_rows_index = list(set(temp_dict.keys()) - set(og_dict))
    new_rows_count = len(new_rows_index) #number of rows to append

    #append new rows 
    lst_of_lst = [[movie, None, None, None, None, None, i,temp_dict[i][0]] for i in new_rows_index]
    df.reset_index(drop=True, inplace=True)
    new_df = df.append(pd.DataFrame(lst_of_lst, columns=df.columns))
    new_df.sort_values(by=['format_date'], inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    
    #add new column with adj values
    new_df['adj_gtd'] = ''

    for index, row in new_df.iterrows():
        new_df.loc[index, 'adj_gtd'] = temp_dict[index+1][1]
    
    
    #Find weekly gross
    new_df = add_weekly_gross(new_df)

    return new_df

In [13]:
new_df_boolean = True

lst = all_titles

for movies in all_titles:
    if movies in lst:
        print("processing for movie " + str(movies))

        temp_df = titles_df.loc[titles_df['title'] == movies]
        movie_df = get_movie_df(movies, temp_df)
        if new_df_boolean is True:
            clean_df = movie_df
            new_df_boolean = False
        else:
            clean_df = pd.concat([clean_df, movie_df], ignore_index=True, sort=True)[clean_df.columns]
            
#clean_df.loc[clean_df['title']== 'Lion']
#clean_df.loc[clean_df['title'] == 'Star Wars: The Last Jedi']

print("number of records we have after data imputation is : " + str(len(clean_df)))
clean_df

processing for movie Incredibles 2
processing for movie Beauty and the Beast
processing for movie The Greatest Showman
processing for movie Black Panther
processing for movie Avengers Infinity War
processing for movie Jurassic World
processing for movie Wonder Woman
processing for movie Star Wars: The Last Jedi
processing for movie Coco
processing for movie Overlord
processing for movie Dunkirk
processing for movie Night School
processing for movie The Grinch
processing for movie The Predator
processing for movie The Meg
processing for movie Occupation
processing for movie Ladies in Black
processing for movie Sweet Country
processing for movie Lion
number of records we have after data imputation is : 359


,title,date_range,year,rank,weekend_gross,gross_to_date,week_no_release,format_date,adj_gtd,shift,week_gross
0,Incredibles 2,Jun. 14–17,2018,1,7898856,7898856,1,2018-06-14,7898856,0.0,7.89886e+06
1,Incredibles 2,Jun. 21–24,2018,2,4868158,14708509,2,2018-06-21,14708509,7898856.0,6.80965e+06
2,Incredibles 2,Jun. 28–Jul. 1,2018,2,3253530,19313351,3,2018-06-28,19313351,14708509.0,4.60484e+06
3,Incredibles 2,Jul. 5–8,2018,2,2940132,24578641,4,2018-07-05,24578641,19313351.0,5.26529e+06
4,Incredibles 2,Jul. 12–15,2018,2,2586378,29501312,5,2018-07-12,29501312,24578641.0,4.92267e+06
...,...,...,...,...,...,...,...,...,...,...,...
354,Lion,Mar. 30–Apr. 2,2017,-,68650,22091664,11,2017-03-30,22091664,21957499.0,134165
355,Lion,Apr. 6–9,2017,-,53044,22181827,12,2017-04-06,22181827,22091664.0,90163
356,Lion,None,None,None,None,None,13,2017-04-13,22246512,22181827.0,64685
357,Lion,Apr. 20–23,2017,-,26430,22311197,14,2017-04-20,22311197,22246512.0,64685


## Process Data 

Get year and week number, to compare against box of performance. Create new column df['year_week']

In [14]:
df = clean_df
df['week_num'] = df['format_date'].apply(lambda x: str(datetime.fromisoformat(x).isocalendar()[1])) 
df['week_num'] = df['week_num'].apply(lambda x: '0'+str(x) if len(x)==1 else str(x)) 
df['year_week'] = df['format_date'].astype(str).str[:4] + "_" + df['week_num']
df.head()

,title,date_range,year,rank,weekend_gross,gross_to_date,week_no_release,format_date,adj_gtd,shift,week_gross,week_num,year_week
0,Incredibles 2,Jun. 14–17,2018,1,7898856,7898856,1,2018-06-14,7898856,0.0,7.89886e+06,24,2018_24
1,Incredibles 2,Jun. 21–24,2018,2,4868158,14708509,2,2018-06-21,14708509,7898856.0,6.80965e+06,25,2018_25
2,Incredibles 2,Jun. 28–Jul. 1,2018,2,3253530,19313351,3,2018-06-28,19313351,14708509.0,4.60484e+06,26,2018_26
3,Incredibles 2,Jul. 5–8,2018,2,2940132,24578641,4,2018-07-05,24578641,19313351.0,5.26529e+06,27,2018_27
4,Incredibles 2,Jul. 12–15,2018,2,2586378,29501312,5,2018-07-12,29501312,24578641.0,4.92267e+06,28,2018_28


## Find Percentage of Weekend Gross 


Create new column to store % of weekend gross. 

Returns irregular values, unable to estimate weekend gross for imputed data

In [15]:
df['weekend_perc'] = df['weekend_gross'].astype(float)/df['week_gross'].astype(float) *100
#df.head(50)

## Find Percentage Change in Gross


Create new column to store % change in gross from previous week

In [16]:
df['perc_change'] = (df['adj_gtd'].astype(float) - df['shift'].astype(float))/df['shift'].astype(float)*100
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.head()

,title,date_range,year,rank,weekend_gross,gross_to_date,week_no_release,format_date,adj_gtd,shift,week_gross,week_num,year_week,weekend_perc,perc_change
0,Incredibles 2,Jun. 14–17,2018.0,1,7898856,7898856,1,2018-06-14,7898856,0.0,7898856.0,24,2018_24,100.000000,NaN
1,Incredibles 2,Jun. 21–24,2018.0,2,4868158,14708509,2,2018-06-21,14708509,7898856.0,6809653.0,25,2018_25,71.489076,86.210623
2,Incredibles 2,Jun. 28–Jul. 1,2018.0,2,3253530,19313351,3,2018-06-28,19313351,14708509.0,4604842.0,26,2018_26,70.654541,31.307334
3,Incredibles 2,Jul. 5–8,2018.0,2,2940132,24578641,4,2018-07-05,24578641,19313351.0,5265290.0,27,2018_27,55.839887,27.262436
4,Incredibles 2,Jul. 12–15,2018.0,2,2586378,29501312,5,2018-07-12,29501312,24578641.0,4922671.0,28,2018_28,52.540135,20.028247


In [17]:
df.to_csv('../Data/new_titles_performance.csv')

## End